In [ ]:
#Sean Poston
#Jon Harsy
#Evan Wilzbach
#Pandas Programming Assignment

In [ ]:
#Brief Statistic
import pandas as pd

summ = pd.read_csv(r'SP500.csv')

print('Summary:')
print(summ[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']].describe())

In [ ]:
#Gain/Loss for April (Dec 2018 doesn't exist)
import pandas as pd
#Switched to April of 2018 because the file stops at April of 2018
#which does not include December of 2018, which is the assignment.

frame = pd.read_csv(r'SP500.csv')
frame['Date'] = pd.to_datetime(frame['Date'])

mask = (frame['Date'] >= '2018-04-1') & (frame['Date'] <= '2018-04-27')

decframe = frame.loc[mask]

fileout = open('aprilgainlossout.csv', 'w')

decframe['Gain/Loss'] = decframe['Close'] - decframe['Open']

fileout.write(decframe.sort_values(['Gain/Loss']).to_csv(index=False))
#We could also remove the Gain/Loss column to keep the data the same with this line instead:
#fileout.write(decframe.sort_values(['Gain/Loss']).drop(['Gain/Loss'], axis=1).to_csv(index=False))

In [ ]:
#Gain above 20% (there are none)
import pandas as pd

frame = pd.read_csv('SP500.csv')

#Mask for daily change over 20%
mask = ((frame['Close'] - frame['Open']) / frame['Open'] >= .2) 

twentygain = frame.loc[mask]

print(twentygain) 
#There are none that are over 20%

In [ ]:
#Highest Daily Gain and Loss
import pandas as pd

frame = pd.read_csv('SP500.csv')

frame['Gain/Loss'] = frame['Close'] - frame['Open']

#Max Gain
print('Max Gain: ')
print(frame[frame['Gain/Loss'] == frame['Gain/Loss'].max()])
#Max Loss
print('\nMax Loss: ')
print(frame[frame['Gain/Loss'] == frame['Gain/Loss'].min()])

In [ ]:
#Highest Daily transaction volume
import pandas as pd
import numpy as np

df = pd.read_csv('SP500.csv')

mask = df['Volume'] == df['Volume'].max()

print(df[mask])

In [ ]:
#Monthly Report
import pandas as pd

def month(row):
  year = row["Date"].split("-")[0]
  month = row["Date"].split("-")[1]
  return year+"-"+month

# grouping function
def gfunc(x):
  d = {}
  d['Total_Volume'] = x['Volume'].sum()
  d['Average_Open_Price'] = x['Open'].mean()
  d['Average_Close_Price'] = x['Close'].mean()
  d['Gain/Loss'] = d['Average_Close_Price'] - d['Average_Open_Price']
  return pd.Series(d, index=['Average_Open_Price', 'Average_Close_Price', 'Total_Volume', 'Gain/Loss'])

df = pd.read_csv("SP500.csv")
# year and month parameters 
df['Year-Month'] = df.apply (lambda row: month(row), axis=1)

# Group based on new column created and the function
groups = df.groupby(['Year-Month']).apply(gfunc)

groups['Date'] = pd.to_datetime(df['Date'])

#query
min_limit = 2666
max_limit = 2800
groups = groups[(groups["Average_Open_Price"] > min_limit) & (groups["Average_Open_Price"] < max_limit)]
print(groups)

In [ ]:
#Yearly Report
import pandas as pd

def getYear(row):
  year = row["Date"].split("-")[0]
  return year

# grouping function
def gfunc(x):
  d = {}
  d['Total_Volume'] = x['Volume'].sum()
  d['Average_Open_Price'] = x['Open'].mean()
  d['Average_Close_Price'] = x['Close'].mean()
  d['Gain/Loss'] = d['Average_Close_Price'] - d['Average_Open_Price']
  return pd.Series(d, index=['Average_Open_Price', 'Average_Close_Price', 'Total_Volume', 'Gain/Loss'])

df = pd.read_csv("SP500.csv")
# year 
df["Year"] = df.apply (lambda row: getYear(row), axis=1)

# Group based on new column created and the function
groups = df.groupby(["Year"]).apply(gfunc)

print(groups)

print('Max Gain Year: ')
print(groups[groups['Gain/Loss'] == groups['Gain/Loss'].max()])

In [ ]:
#5-Year Report
import pandas as pd
import numpy as np
np.set_printoptions(formatter={'float': lambda x: "{0:13.2f}".format(x)})
"""
Array Postitions
0 - Date
1 - Open
2 - High
3 - Low
4 - Close
5 - Adj Close
6 - Volume
"""

def generator(file):
    file.readline()
    for row in file:
        ret = row.split(',')
        ret[-1] = ret[-1].rstrip('\n')
        yield np.asarray(ret)


if __name__ == '__main__':
    file = open('SP500.csv', 'r')
    gen = generator(file)
    
    
    segYear = 0
    averages = []
    """
    [year][0] - open
    [year][1] - close
    [year][2] - vol
    [year][3] - gain/loss
    """

    count = 0
    curOpen = 0.0
    curClose = 0.0
    fluct = 0.0 # gain/loss
    curVol = 0


    for row in gen:
        currentYear = int(row[0][0:4])
        if segYear == 0:
            segYear = currentYear
        if currentYear > segYear + 4 or row[0] == '2018-04-27':
            #averages
            averages.append([curOpen / count, curClose / count, 
                curVol / count, fluct / count]) 
            #reset numbers
            count = 0
            curOpen = float(row[1])
            curClose = float(row[4])
            curVol = int(row[6])
            fluct = 0
            segYear += 5
        else:
            count += 1
            curOpen += float(row[1])
            curClose += float(row[4])
            curVol += int(row[6])
            fluct += float(row[4]) - float(row[1])

    for i in range(len(averages)):
        averages[i].insert(0, 1950 + i * 5)
    averages = np.array(averages)
    print(f'{"Year":>15}{"Open":>14}{"Close":>14}{"Volume":>14}{"Gain/Loss":>14}')
    print(averages) #print out averages of each column for each year

    maxAvgProfit = averages[0][4]
    mostProfitYear = averages[0][0]

    for row in averages:
        if row[4] > maxAvgProfit:
            maxAvgProfit = row[4]
            mostProfitYear = row[0]

    print(f'Most profitable year: {mostProfitYear}')